# Libraries and Configuration

In [2]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import (mean_squared_error, r2_score,
                             mean_poisson_deviance,
                             mean_absolute_error, accuracy_score)
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import KFold
import requests
from datetime import datetime
from tqdm.notebook import tqdm
import nmap
from scipy.spatial import distance as dist
from imutils import build_montages
import cv2
import pickle
import time
from datetime import datetime as dt
import tensorflow as tf

import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('max_colwidth', 80)

# import drawing images
from IPython.display import Image

# make sure images are centered and not full screen
from IPython.core.display import HTML
HTML("""
<style>
.jp-OutputArea-output {
    display: table-cell;
    text-align: center !important;
    vertical-align: middle !important;
}
.jp-OutputArea-output img {
    width: 70% !important;
}
</style>
""")

import matplotlib.pyplot as plt
from pylab import rcParams
import seaborn as sns
%matplotlib inline
sns.set_style('whitegrid')

import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
from plotly.subplots import make_subplots
pio.templates.default = 'plotly_white'  # 'plotly' | 'plotly_dark'
pio.renderers.default = "jupyterlab"  # 'svg' | 'jupyterlab'

In [6]:
CONFIG = {
    'IMG_BASE_DIR': '/data/security_cam_detections_v2/Front-Parking',
    'DOWNTIME_DATES': ['2020-01-13', '2020-01-14', '2020-02-28'],
    'RANDOM_STATE': 42
}

In [2]:
# GPU code only, skip if no GPU is available!
# make sure we don't run out of memory by running multiple Notebooks together
# https://www.tensorflow.org/api_docs/python/tf/config/experimental/set_memory_growth
physical_devices = tf.config.list_physical_devices('GPU') 
try:
    # print('TF -> Using GPU -> ', physical_devices[0])
    tf.config.experimental.set_memory_growth(physical_devices[0], True) 
except Exception as e: 
    # Invalid device or cannot modify virtual devices once initialized. 
    print('GPU Error, if you have no GPU it is not a problem', e)
    pass

In [3]:
# define reusable functions here
def plt_loss(train_loss, val_loss):
    """
    Construct the multi scatter plot to show training and validation loss curves
    Parameters
    ----------
    train_loss : list, numpy array or similar
    val_loss : list, numpy array or similar
    """
    fig = go.Figure()
    fig.add_trace(go.Scatter(y=train_loss, name='Training Loss', mode='lines'))
    fig.add_trace(go.Scatter(y=val_loss, name='Validation Loss', mode='lines'))
    fig.update_layout(title='Training Loss', height=375, width=600, xaxis_title="Epoch",
                      yaxis_title="Loss")
    fig.show()